In [2]:
import numpy as np
import pandas as pd

##preproc sip

In [21]:
#PATH
sip_train = pd.read_csv('./sip_train.csv')
sip_test = pd.read_csv('./sip_test.csv')

In [22]:
sip_train.head()

,NPLV,VDSYP,NMSYP,VSSYP,DAT_OTD
0,510008,346,Уголь ТО,570,2021-01-01 03:03:53
1,510008,346,Уголь ТО,220,2021-01-01 03:04:10
2,510008,408,изв_ЦОИ,7300,2021-01-01 03:08:17
3,510008,346,Уголь ТО,270,2021-01-01 03:09:03
4,510008,346,Уголь ТО,430,2021-01-01 03:09:20


In [23]:
print(sip_train.NMSYP.value_counts(), "\n")
print(sip_test.NMSYP.value_counts())

изв_ЦОИ       19055
Уголь ТО       7747
ФЛЮМАГ         2930
Флюс ФОМИ      1624
изв отсев       226
Долом(сыр)        2
Name: NMSYP, dtype: int64 

изв_ЦОИ       7718
Уголь ТО      1049
ФЛЮМАГ         842
Флюс ФОМИ      341
изв_отсев      204
агл_офл_с      154
долом_сыр      147
кокс_25_40       7
Name: NMSYP, dtype: int64


In [24]:
sip_train.drop('VDSYP', axis=1, inplace=True)
sip_test.drop('VDSYP', axis=1, inplace=True)
sip_test['NMSYP'] = sip_test.NMSYP.apply(lambda x: "изв отсев" if x == "изв_отсев" else x)

NMSYP_top_values = sip_test.NMSYP.value_counts().index.to_list()[0:5]
NMSYP_top_values

['изв_ЦОИ', 'Уголь ТО', 'ФЛЮМАГ', 'Флюс ФОМИ', 'изв отсев']

In [25]:
def get_sip_df(df_base: pd.DataFrame, NMSYP_values: list) -> pd.DataFrame:
    '''
        df_base: датафрейм sip_train или sip_test
        result: датафрейм df_finally с признаками суммы добавочных веществ.
    '''

    # Датафрейм из номеров плавки
    df_finally = pd.DataFrame({"NPLV": df_base['NPLV'].unique()})

    # Группировка по номеру плавки и названию сыпучей добавки
    grouper = df_base.groupby(['NPLV', 'NMSYP'], as_index=False)

    # пробегаемся по всем значениям NMSYP. Для каждого значения номера плавка получаем сумму по значению NMSYP
    mass_sum = grouper['VSSYP'].sum()
    operations_count = grouper['VSSYP'].count()

    for nmsyp in NMSYP_values:
        df_nmsyp = mass_sum[mass_sum.NMSYP == nmsyp].reset_index(drop=True).drop('NMSYP', axis=1).rename(columns={"VSSYP": nmsyp + "_sum"})
        df_operations = operations_count[operations_count.NMSYP == nmsyp].reset_index(drop=True).drop('NMSYP', axis=1).rename(columns={"VSSYP": nmsyp + "_op_count"})
        
        df_finally = pd.merge(df_finally, df_nmsyp, on=('NPLV'), how='left')
        df_finally = pd.merge(df_finally, df_operations, on=('NPLV'), how='left')
    
    df_finally = df_finally.fillna(0)

    return df_finally

In [26]:
df_sip_train = get_sip_df(sip_train, NMSYP_top_values)
df_sip_test = get_sip_df(sip_test, NMSYP_top_values)

In [27]:
df_sip_train.head()

,NPLV,изв_ЦОИ_sum,изв_ЦОИ_op_count,Уголь ТО_sum,Уголь ТО_op_count,ФЛЮМАГ_sum,ФЛЮМАГ_op_count,Флюс ФОМИ_sum,Флюс ФОМИ_op_count,изв отсев_sum,изв отсев_op_count
0,510008,14080,5,2950.0,5.0,2960.0,1.0,980.0,1.0,0.0,0.0
1,510009,18830,8,2930.0,5.0,0.0,0.0,960.0,1.0,1060.0,1.0
2,510010,16080,5,2990.0,5.0,2960.0,1.0,1050.0,1.0,990.0,1.0
3,510011,22150,6,3620.0,5.0,0.0,0.0,980.0,1.0,550.0,1.0
4,510012,19550,8,2930.0,3.0,3010.0,2.0,1000.0,1.0,2050.0,2.0


In [28]:
df_sip_test.head()

,NPLV,изв_ЦОИ_sum,изв_ЦОИ_op_count,Уголь ТО_sum,Уголь ТО_op_count,ФЛЮМАГ_sum,ФЛЮМАГ_op_count,Флюс ФОМИ_sum,Флюс ФОМИ_op_count,изв отсев_sum,изв отсев_op_count
0,512324,13960,10,1310.0,4.0,1670.0,1.0,0.0,0.0,0.0,0.0
1,512327,15290,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,512328,20010,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,512331,14060,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,512333,20100,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#df_sip_train.to_csv("preproc_sip_train.csv", index=False)
#df_sip_test.to_csv("preproc_sip_test.csv", index=False)

##preproc chronom


In [31]:
#PATH
chronom_train = pd.read_csv("chronom_train.csv")
chronom_test = pd.read_csv("chronom_test.csv")

In [32]:
chronom_train.head()

,Unnamed: 0,NPLV,TYPE_OPER,NOP,VR_NACH,VR_KON,O2
0,35171,510008,межпл.прост.,межпл. простой,2011-01-01 10:28:13,2011-01-01 10:32:15,NaN
1,35162,510008,межпл.прост.,межпл. простой,2021-01-01 03:01:07,2021-01-01 03:08:11,NaN
2,35163,510008,межпл.прост.,Осмотр конвертера,2021-01-01 03:01:19,2021-01-01 03:03:43,NaN
3,35164,510008,межпл.прост.,Наведение гарнисажа,2021-01-01 03:03:43,2021-01-01 03:05:23,NaN
4,35165,510008,межпл.прост.,Слив шлака,2021-01-01 03:05:23,2021-01-01 03:06:09,NaN


In [33]:
chronom_test.head()

,Unnamed: 0,NPLV,TYPE_OPER,NOP,VR_NACH,VR_KON,O2
0,0,512324,межпл.прост.,межпл. простой,2011-05-07 21:28:59,2011-05-07 21:33:38,NaN
1,1,512324,межпл.прост.,межпл. простой,2021-05-05 17:35:55,2021-05-05 17:41:21,NaN
2,2,512324,межпл.прост.,Осмотр конвертера,2021-05-05 17:36:07,2021-05-05 17:39:07,NaN
3,3,512324,межпл.прост.,Слив шлака,2021-05-05 17:39:07,2021-05-05 17:41:20,NaN
4,4,512324,опер,Завалка лома,2021-05-05 17:41:21,2021-05-05 17:43:16,NaN


In [34]:
def preproc_base(df_base: pd.DataFrame):
    '''
    df_base: chronom_train.csv или chronom_test.csv
    '''
    # 1) удаляем ненужный столбец (сильно коррелирует с NOP - названием операции)
    # 2) удаляем столбец Unnamed с лишними индексами
    df_base.drop('Unnamed: 0', axis=1, inplace=True)
    df_base = df_base[df_base['TYPE_OPER'] == 'опер'].reset_index(drop=True)
    df_base.drop('TYPE_OPER', axis=1, inplace=True)

    # преобразуем время
    df_base['VR_NACH'] = pd.to_datetime(df_base['VR_NACH'])
    df_base['VR_KON'] = pd.to_datetime(df_base['VR_KON'])

    # заполнение пропущенных значений
    index_op_with_na = list(df_base[df_base.VR_KON.isna()].index)
    for i in index_op_with_na:
        df_base.loc[i, 'VR_KON'] = df_base.loc[i + 1, 'VR_NACH']

    # Исправляем строки в которых время начала операции больше времени конца
    index_error_time = list(df_base[df_base.VR_NACH > df_base.VR_KON].index)
    for i in index_error_time:
        df_base.loc[i, 'VR_KON'] = df_base.loc[i + 1, 'VR_NACH']

    # Добавляем столбец с длительностью операций
    df_base['duration'] = (df_base.VR_KON - df_base.VR_NACH).astype('timedelta64[s]')

    # убираем ненужные столбцы
    df_base.drop('VR_NACH', axis=1, inplace=True)
    df_base.drop('VR_KON', axis=1, inplace=True)
    df_base.drop('O2', axis=1, inplace=True)

    return df_base

In [35]:
chronom_test = preproc_base(chronom_test)
chronom_train = preproc_base(chronom_train)

In [36]:
chronom_train.head()

,NPLV,NOP,duration
0,510008,Завалка лома,16.0
1,510008,Нагрев лома,365.0
2,510008,Нагрев лома,365.0
3,510008,Заливка чугуна,203.0
4,510008,Продувка,1170.0


In [37]:
def get_chronom_dur(df_base: pd.DataFrame) -> pd.DataFrame:
    '''
        df_base: датафрейм chronom_train или chronom_test
        result: датафрейм df_finally, который содержит длительность операций
                для каждого номера плавки.
    '''

    # Датафрейм из номеров плавки
    df_finally = pd.DataFrame({"NPLV": df_base['NPLV'].unique()})

    # Группировка по номеру плавки и названию операции
    grouper = df_base.groupby(['NPLV', 'NOP'], as_index=False)

    # пробегаемся по всем значениям NOP. Для каждого значения номера плавка получаем сумму по значению duration
    dur_sum = grouper['duration'].sum()
    #NOP_values = df_base.NOP.unique() # все значения NOP
    #NOP_values.sort()

    # Только те операции, которые присутствуют в тестовых данных
    NOP_values = ['Завалка лома', 'Нагрев лома', 'Заливка чугуна', 'Продувка'] 
    for name_op in NOP_values:
        df_nop = dur_sum[dur_sum.NOP == name_op].reset_index(drop=True).drop('NOP', axis=1).rename(columns={"duration": name_op + "_duration"})
        df_finally = pd.merge(df_finally, df_nop, on=('NPLV'), how='left')
    
    df_finally = df_finally.fillna(0)

    return df_finally

In [38]:
chronom_dur_train = get_chronom_dur(chronom_train)
chronom_dur_test = get_chronom_dur(chronom_test)

In [39]:
chronom_dur_train.head()

,NPLV,Завалка лома_duration,Нагрев лома_duration,Заливка чугуна_duration,Продувка_duration
0,510008,16.0,730.0,203.0,1170.0
1,510009,31.0,788.0,188.0,1230.0
2,510010,133.0,738.0,209.0,1226.0
3,510011,17.0,776.0,175.0,1058.0
4,510012,136.0,736.0,184.0,1239.0


In [40]:
chronom_dur_test.head()

,NPLV,Завалка лома_duration,Нагрев лома_duration,Заливка чугуна_duration,Продувка_duration
0,512324,115.0,730.0,143.0,1263.0
1,512327,114.0,0.0,578.0,1074.0
2,512328,2096.0,0.0,150.0,1122.0
3,512331,3015.0,0.0,221.0,1103.0
4,512333,1478.0,0.0,199.0,1096.0


##MERGE ALL TABLES

In [ ]:
chronom_harry_train = pd.read_csv('df_chronom_train.csv')
chronom_harry_test = pd.read_csv('df_chronom_test.csv')

# Удаляем столбцы, которые в тестовых данных полностью заполнены нулями
chronom_harry_train.drop('O2_C', axis=1, inplace=True)
chronom_harry_train.drop('O2_t', axis=1, inplace=True)
chronom_harry_test.drop('O2_C', axis=1, inplace=True)
chronom_harry_test.drop('O2_t', axis=1, inplace=True)

chronom_complete_train = pd.merge(chronom_harry_train, chronom_dur_train, on=('NPLV'))
chronom_complete_test = pd.merge(chronom_harry_test, chronom_dur_test, on=('NPLV'))

In [ ]:
# подгружаем таблицу produv + gas + plavki + sip
train_all = pd.read_csv("train.csv")
test_all = pd.read_csv("test.csv")

# chugun preproc table
chugun_train = pd.read_csv("chugun_train_csv.csv")
chugun_test = pd.read_csv("chugun_test_csv.csv")

# lom preproc table
lom_train = pd.read_csv("lom_train_csv_1.csv")
lom_test = pd.read_csv("lom_test_csv_1.csv")

In [ ]:
# соединяем в одну таблицу
train_all = pd.merge(train_all, chronom_final_train, on=('NPLV'))
test_all = pd.merge(test_all, chronom_final_test, on=('NPLV'))

train_all = pd.merge(train_all, chugun_train, on=('NPLV'))
test_all = pd.merge(test_all, chugun_test, on=('NPLV'))

train_all = pd.merge(train_all, lom_train, on=('NPLV'))
test_all = pd.merge(test_all, lom_test, on=('NPLV'))

In [ ]:
train_all.to_csv("train_all.csv", index=False)
test_all.to_csv("test_all.csv", index=False)

##Стандартизация

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

tt = train_all.append(test_all)
scaler = MinMaxScaler()
scaler.fit(tt)
train_all[:] = scaler.transform(train_all)
test_all[:] = scaler.transform(test_all)

## catboost


In [ ]:
import os
from time import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
!pip install catboost
from catboost import CatBoostRegressor

from sklearn.model_selection import GridSearchCV

In [ ]:
target = pd.read_csv('target_train.csv', index_col='NPLV').fillna(0)
nplv = test_all.NPLV
train_all.drop('NPLV', axis=1, inplace=True)
test_all.drop('NPLV', axis=1, inplace=True)

In [ ]:
# features - таблица с названиями 40 лучших фичей, отбор с помощью random forest
features = pd.read_csv("features.csv")
best_features = features.cols[:40].to_list()

train_all_best = train_all.loc[:, best_features]
test_all_best = test_all.loc[:, best_features]

In [ ]:


X_train_tst, X_test_tst, y_train_tst, y_test_tst = train_test_split(train_all_best, target['TST'], test_size=0.2, random_state=42)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(train_all_best, target['C'], test_size=0.2, random_state=42)
cat_c = CatBoostRegressor(depth=7, iterations=2000, learning_rate=0.03, task_type='GPU', random_seed=42)
cat_tst = CatBoostRegressor(depth=7, iterations=2000, learning_rate=0.03, task_type='GPU', random_seed=42)

# param_grid = {'depth' : [3, 7, 9],
#                 'learning_rate' : [0.03, 0.05, 0.1],
#                 'l2_leaf_reg' : [3, 5, 7]}
# best_cat_c = GridSearchCV(cat_c, param_grid=param_grid, n_jobs=-1, verbose=1, cv=2).fit(X_train_c, y_train_c).best_estimator_
# best_cat_ts = GridSearchCV(cat_ts, param_grid=param_grid, n_jobs=-1, verbose=1, cv=2).fit(X_train_tst, y_train_tst).best_estimator_

cat_c.fit(X_train_c, y_train_c)
cat_tst.fit(X_train_tst, y_train_tst)

In [ ]:
preds_tst = cat_tst.predict(test_all)
preds_c = cat_c.predict(test_all)
  
submit = pd.DataFrame({'NPLV' : nplv, 'TST' : preds_tst, 'C' : preds_c})
submit.to_csv(f'submission_{time()}.csv')

##LGBM

In [ ]:
!pip install optuna

In [ ]:
import lightgbm as lgbm
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_squared_error
sampler = TPESampler(seed=13)

def create_model(trial):
    num_leaves = trial.suggest_int("num_leaves", 2, 80)
    n_estimators = trial.suggest_int("n_estimators", 50, 500)
    max_depth = trial.suggest_int('max_depth', 2, 7)
    min_child_samples = trial.suggest_int('min_child_samples', 2, 100)
    learning_rate = trial.suggest_uniform('learning_rate', 0.001, 0.99)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 2, 50)
    feature_fraction = trial.suggest_uniform('feature_fraction', 0.00001, 1.0)
    
    model = lgbm.LGBMRegressor(
        num_leaves=num_leaves,
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        min_child_samples=min_child_samples, 
        min_data_in_leaf=min_data_in_leaf,
        learning_rate=learning_rate,
        feature_fraction=feature_fraction,
        random_state=13,
        n_jobs=-1
)
    return model

def objective(trial):
    model = create_model(trial)
    X_train, X_test, y_train, y_test = train_test_split(train_all_best, target['TST'])
    model.fit(X_train, y_train)
    result = model.predict(X_test)
    score = mean_squared_error(y_test.values, result)
    return score

# раскоментить надо последние 3 строчки для запуска оптимизатора

# study = optuna.create_study(direction="minimize", sampler=sampler)
# study.optimize(objective, n_trials=1000)
# params_lgbm = study.best_params

In [ ]:
# параметры, полученные с помощью оптюны
params_lgbm_C = {'feature_fraction': 0.9128015319322991,
                'learning_rate': 0.16001167282336393,
                'max_depth': 2,
                'min_child_samples': 17,
                'min_data_in_leaf': 24,
                'n_estimators': 145,
                'num_leaves': 4}

params_lgbm_tst = {'feature_fraction': 0.4632561843030582,
                'learning_rate': 0.09810862221017563,
                'max_depth': 6,
                'min_child_samples': 82,
                'min_data_in_leaf': 41,
                'n_estimators': 183,
                'num_leaves': 16}

In [ ]:
lgbm_c = lgbm.LGBMRegressor(**params_lgbm_tst)
lgbm_tst = lgbm.LGBMRegressor(**params_lgbm_tst)

lgbm_c.fit(train_all_best, target['C'])
lgbm_tst.fit(train_all_best, target['TST'])

In [ ]:
preds_tst = lgbm_tst.predict(test_all_best)
preds_c =  lgbm_c.predict(test_all_best)

submit = pd.DataFrame({'NPLV' : nplv, 'TST' : preds_tst, 'C' : preds_c})
submit.to_csv(f'submission_{time()}.csv')